In [35]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier

In [36]:
def chunk_filter_data(X,Y,chunk_size,train=False,stride=1):
    X_out = []
    Y_out = []
    for i in range(0,len(X),stride):
        x = X[i:i+chunk_size]
        y = Y[i:i+chunk_size]
        if x.shape[0] != chunk_size:
            continue
        std = np.std(x,axis=0).shape
        fvec = x.flatten()
#         fvec = std
        if train:
            mean = np.mean(y)
            if mean > 0.5:
                Y_out.append(1)
                X_out.append(fvec)
            elif mean < 0.5:
                Y_out.append(0)
                X_out.append(fvec)
        else:
            if (np.all(y == y[0])) == False:
                continue
            Y_out.append(y[0])
            X_out.append(fvec)
    return np.array(X_out), np.array(Y_out)

In [37]:
df = pd.read_pickle("./calibdata.pkl")

In [38]:
continous = np.mean(np.array(df["Gaze Target XY"].values.tolist()),axis=1)
fixation = [0]
for i in range(1,len(continous)):
    fixation.append(continous[i] - continous[i-1])
for i in range(0,len(fixation)):
    if fixation[i] == 0:
        fixation[i] = True
    else:
        fixation[i] = False
df['fixation'] = fixation

In [39]:
feat = "Embeddings Hist"
control = "Subject"
df_group = df.groupby([control])["fixation",feat]
g_id = list(df_group.groups.keys())

In [40]:
chunk_size = 15

Y_OUT, Y_GT = None, None
acc = []

for i in range(len(g_id)):
    print("Participant",g_id[i])
    X_train, Y_train, X_test, Y_test = None, None, None, None
    
    for j in range(len(g_id)):
        
        df_extract = df_group.get_group(g_id[j])
        x = np.array(df_extract[feat].tolist())
        y = np.array(df_extract["fixation"].tolist())
        x = x.reshape(x.shape[0],x.shape[1]*x.shape[2])
        y = y.reshape(-1,1)
        if g_id[i] == g_id[j]:
            X_test = x
            Y_test = y
        else:
            if X_train is None:
                X_train = x
                Y_train = y
            else:
                X_train = np.vstack((X_train,x))
                Y_train = np.vstack((Y_train,y))
        
    
    X_train = np.nan_to_num(X_train)
    X_test = np.nan_to_num(X_test)
    
    X_train, Y_train = chunk_filter_data(X_train,Y_train,chunk_size,train=True)
    X_test, Y_test = chunk_filter_data(X_test,Y_test,chunk_size,train=True)

    print("\t Train shape:",X_train.shape,Y_train.shape)
    print("\t Test shape:",X_test.shape,Y_test.shape)
    
    min_max_scaler_x = preprocessing.MinMaxScaler()
    X_train = min_max_scaler_x.fit_transform(X_train)
    X_test = min_max_scaler_x.transform(X_test)
    
#     clf = ExtraTreesClassifier(n_estimators=100,class_weight='balanced')
    clf = MLPClassifier()
    
    clf.fit(X_train,Y_train)
    
    Y_pred = clf.predict(X_test).reshape(-1,1)
    
    if Y_OUT is None:
        Y_OUT = Y_pred
        Y_GT = Y_test
    else:
        Y_OUT = np.vstack((Y_OUT,Y_pred))
        Y_GT = np.vstack((Y_GT,Y_test))
    
    print("\t Accuracy",accuracy_score(Y_test,Y_pred))
    acc.append(accuracy_score(Y_test,Y_pred))


Participant 0
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (1386,)
	 Accuracy 0.7727272727272727
Participant 1
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (1386,)
	 Accuracy 0.7366522366522367
Participant 2
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (1386,)
	 Accuracy 0.7655122655122655
Participant 3
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (1386,)
	 Accuracy 0.6536796536796536
Participant 4
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (1386,)
	 Accuracy 0.759018759018759
Participant 5
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (1386,)
	 Accuracy 0.6428571428571429
Participant 6
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (1386,)
	 Accuracy 0.6572871572871573
Participant 7
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (1386,)
	 Accuracy 0.7987012987012987
Participant 8
	 Train shape: (12586, 390) (12586,)
	 Test shape: (1386, 390) (138

In [41]:
print("Accuracy",np.mean(acc),"SD",np.std(acc))

Accuracy 0.7253246753246754 SD 0.05915518372248136


In [42]:
print("Baseline acc",np.mean(Y_test))

Baseline acc 0.5721500721500722
